# Start

In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import pyodbc

from pat_back import PsoldRating , PAT_FLAG, VALIDATE_RULE, COVERAGE_TYPE, PERIL_SUBGROUP, DEDDUCT_TYPE, RATING_TYPE, PSOLD_PERSP

fld = r"C:\Users\cxiao\source\repos\property-exposure\gcpropexpo\GCPropExpo\tests\testthat"
df_loc = pd.read_csv(f'{fld}/Inputs/PATLocations.csv')
df_loc['LOCID'] = df_loc.index +1
df_pol = pd.read_csv(f'{fld}/Inputs/PATPolicies.csv')
df_res = pd.read_csv(f'{fld}/Controls/PAT_Ans.csv')

DT = df_pol.merge(df_loc, on='Policy ID')
DT.columns=['PolicyID', 'Limit', 'Retention', 'PolPrem', 'LossRatio', 'Participation', 'TIV', 'Stack', 'RatingGroup', 'LOCID']



### Input Parameters

In [2]:
# # No HPR PSOLD 2020
# test_that("Accuracy", {
#   loc <- fread('./Inputs/PATLocations.csv')
#   pol <- fread('./Inputs/PATPolicies.csv')
#   covgInfo <- list(subPrem = 1e6,
#                    ratio = 0.5,
#                    covg = 4L,
#                    subgrp = 4L,
#                    dedType = 1L,
#                    curAdj = 1,
#                    avAccDt = "1/1/2023",
#                    blendHPR = FALSE,
#                    ISOt = "PSOLD_2020",
#                    curveType = 'gross',
#                    type = 'psold',
#                    occupancyWeights = c(rep(0, 38L), 1))
#   correctAns <- masterAns[TEST == 'PSOLD2020_1', .(LOCID, PREMIUM)]
#   testResults <- PAT(pol, loc, covgInfo)
#   expect_equal(testResults$allocPrem, correctAns$PREMIUM, tolerance = tol)
# })


ratio = 0.5
covg = COVERAGE_TYPE['Building_Contents_BI']
adcvg = 2
subg = PERIL_SUBGROUP['All_Perils']
dedType = DEDDUCT_TYPE['Retains_Limit']
curAdj = 1
avAccDt = datetime(2023,1,1)
blendHPR = False
ISOt = 2020
curveType = PSOLD_PERSP['Gross']
blending = np.concatenate((np.zeros(38), np.ones(1)))
test = 'PSOLD2020_1'

DT.fillna({'LossRatio': ratio}, inplace=True)
if np.any(blending>0):
    if np.sum(blending>0)==1:
        def_rg = np.argmax(blending>0) + 1
        DT.fillna({'RatingGroup':def_rg}, inplace=True)
        blending = None
else:
    blending = None


### Curve data

In [3]:
conn_str = f'''DRIVER={{SQL Server}};Server=dfwcat-pm;Database=premium_allocation_tool;
            User Id=sa;Password=In5tr@t1;MultipleActiveResultSets=true;'''
df_wts, df_hpr = None, None
with pyodbc.connect(conn_str) as conn:
    aoi_split = pd.read_sql_query(
        f"""select * from psold_aoi order by AOI""", conn).AOI.to_numpy()
    df_psold = pd.read_sql_query(f"""select * from psold_curves  
        where ID = {ISOt} and CurveType = '{curveType.name}' 
            and COVG = {covg} 
            and SUBGRP = {subg}""", conn).drop(columns=['ID', 'CurveType', 'COVG', 'SUBGRP'])

    if blending is not None:
        df_wts = pd.read_sql_query(f"""select RG, HPRTable from psold_weight order by rg""", conn)
        df_wts['PremiumWeight'] = blending

        if blendHPR:
            df_hpr = pd.read_sql_query(f"""select Limit, Weight from psold_hpr_weight order by Limit""", conn)
        else:
            df_wts.drop(columns=['HPRTable'])


### Allocation

In [4]:
rating_model = PsoldRating(ISOt, df_psold, aoi_split )
DT = rating_model.calculate_las(DT, None, df_wts, df_hpr, 
            ded_type = dedType.name, 
            avg_acc_date = avAccDt,
            addt_cvg = adcvg)

DT['Premium'] = (DT.PolLAS-DT.DedLAS) * DT.LossRatio
sumLAS = DT.groupby('PolicyID').agg( 
    {'Premium': 'sum'}).rename(columns={'Premium': 'sumLAS'})
DT = DT.merge(sumLAS, on='PolicyID')
DT['Premium'] *= DT['PolPrem'] / DT['sumLAS']

### Compare

In [5]:
df = df_res[df_res.TEST== test].merge(DT, on ='LOCID')[['LOCID','PREMIUM','Premium']]
df['Diff']= np.abs(df.PREMIUM-df.Premium) /df.PREMIUM
print('Accuracy: ', df.Diff.max())
df

Accuracy:  3.866776125464571e-05


,LOCID,PREMIUM,Premium,Diff
0,1,2.108379e+05,2.108379e+05,5.783240e-09
1,2,1.854673e+05,1.854673e+05,7.681651e-09
2,3,7.278983e+05,7.278978e+05,7.611361e-07
3,4,0.000000e+00,0.000000e+00,NaN
4,5,8.269248e+04,8.269253e+04,5.778562e-07
5,6,1.814281e+06,1.814252e+06,1.599606e-05
6,7,4.592322e+06,4.592198e+06,2.712345e-05
7,8,3.374436e+05,3.374567e+05,3.866776e-05
8,9,9.683031e+03,9.683029e+03,2.087075e-07
9,10,2.000000e+05,2.000000e+05,0.000000e+00
